# Projet maison n° 3

In [3]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [3]:
import requests
import zipfile
URL = "https://www.ssa.gov/oact/babynames/names.zip"
response = requests.get(URL)
open("name.zip", "wb").write(response.content)


with zipfile.ZipFile("name.zip","r") as zip_ref:
    zip_ref.extractall("./")

In [ ]:
# names us
def df_names_us():
    df = pd.DataFrame()
    return df

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [4]:
import zipfile

import requests

def download_data(url, filename):
    try:
        response = requests.get(url)
        open(filename, "wb").write(response.content)
        return True
    except:
        return False

def unzip_data(filename):
    try:
        with zipfile.ZipFile(filename,"r") as zip_ref:
            zip_ref.extractall("./data/")
        return True
    except:
        return False


URL = "https://www.ssa.gov/oact/babynames/names.zip"
response = requests.get(URL)
open("us_name.zip", "wb").write(response.content)


with zipfile.ZipFile("us_name.zip","r") as zip_ref:
    zip_ref.extractall(r"./data/us_name")
    

french_data_url = "https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip"

download_data(french_data_url, "french_name.zip")
unzip_data("french_name.zip")

False

In [5]:
from os import listdir
from os.path import isfile, join

def list_file(path = "./data/us_name"):
    
    us_name_path = path
    files = [f for f in listdir(us_name_path) if isfile(join(us_name_path, f))]

    return files

In [63]:

from cmath import nan
from unicodedata import name
from jack import Status
from nbformat import read
import pandas as pd
import re

from sqlalchemy import null
def read_files(file_name, us_name_path):
    year =re.findall(r'\d+', file_name)
    try:
        df = pd.read_csv(us_name_path+'/'+ file_name, sep=",",on_bad_lines='skip')
        df.columns = ['name', 'gender', 'births']
        
    
        
        df['year'] = year * len(df)
        
        return df , True
    except:
        data = {'name': None, 'gender': None, 'births': None, 'year': year}
        return pd.DataFrame(data) , False



def df_names_us(path = r"./data/us_name", columns_order = ["year", "name", "gender", "births"]):
    total= pd.DataFrame()


    for file in list_file(path=path):
        data,Status = read_files(file, path)
        total = pd.concat([total, data], axis=0)

    total = remove_short_name(df=total, min_length=2)
    total = column_capitalize(total, col="name")
    
    total = total.dropna()

    total =total.reset_index(drop=True)
    
        #  - Mêmes dtypes pour les colonnes
    total ['year'] = pd.to_datetime (total ['year'] )
    total ['births'] = pd.to_numeric (total ['births'] )
    
    total = total.sort_values(by=['year', 'gender', 'births','name'],
               ascending=[False, False, True,False])
    total =total.reset_index(drop=True)
    return total [columns_order]



def remove_short_name(df, min_length = 2):
    df = df[df.name.str.len() >= min_length]
    return df

def column_capitalize(datafram ,col = "name"):
    datafram[col] = datafram[col].str.capitalize()
    return datafram

# df1 = read_files(onlyfiles[0], us_name_path)
# df2 = read_files(onlyfiles[1], us_name_path)
# df3 = read_files(onlyfiles[2], us_name_path)
# all_row_pd = pd.concat([df1, df2, df3], axis=0)
# all_row_pd

us_data = df_names_us()

In [64]:
us_data

,year,name,gender,births
0,2021-01-01,Zyn,M,5
1,2021-01-01,Zylar,M,5
2,2021-01-01,Zyian,M,5
3,2021-01-01,Zyel,M,5
4,2021-01-01,Zyeire,M,5
...,...,...,...,...
2052634,1880-01-01,Margaret,F,1578
2052635,1880-01-01,Minnie,F,1746
2052636,1880-01-01,Elizabeth,F,1939
2052637,1880-01-01,Emma,F,2003


In [75]:
# names fr



def df_names_fr():
    
    french_data = pd.read_csv(r"./data/french_name/nat2021.csv",  sep = ';',on_bad_lines='skip')

    # - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
    french_data = french_data.rename({'preusuel': 'name', 'annais': 'year', 'sexe':'gender', 'nombre': 'births'}, axis='columns')


    columns_order = ["year", "name", "gender", "births"]
    french_data = french_data [columns_order]




    #  - Mêmes valeurs pour la colonne "gender"
    def convert_gender_label(number):
        if number == 1:
            return "M"
        else:
            return "F"


    french_data['gender'] = french_data ['gender'].apply(convert_gender_label)

    #  - Seuls les prénoms de 2 caractères et plus sont conservés
    french_data = remove_short_name(df=french_data, min_length=2)

    #  - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
    french_data = column_capitalize(french_data, col="name")

    #french_data['year'].unique() . I used this  to find out if there is any year that is not number
    #  - Les lignes avec des données inutilisables doivent être supprimées

    none_accepted_year_format = ['XXXX'] 
    none_accepted_name_format = ['Inconnu', 'Inconnue','_PRENOMS_RARES','_prenoms_rares'] 
    damaged_year_data = french_data['year'].isin(none_accepted_year_format) #
    damaged_name_data = french_data['name'].isin(none_accepted_name_format)
    french_data = french_data [~damaged_year_data]
    french_data = french_data [~damaged_name_data]

    french_data = french_data.dropna()





    #  - Mêmes dtypes pour les colonnes
    french_data['year'] = pd.to_datetime(french_data['year'] )

    french_data['births'] = pd.to_numeric(french_data['births'] )





    #Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
    french_data = french_data.sort_values(by=['year', 'gender', 'births','name'],
                ascending=[False, False, True,False])

    # - L'index du DataFrame doit aller de 0 à N-1
    french_data =french_data.reset_index(drop=True)
    return french_data

In [53]:
us_data.dtypes

year      object
name      object
gender    object
births    object
dtype: object

In [52]:
french_data.dtypes

year      object
name      object
gender    object
births     int64
dtype: object

In [58]:
us_data.dtypes == french_data.dtypes

year      True
name      True
gender    True
births    True
dtype: bool

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [126]:
# taux de change


us_dollar_data_file = r'./data/us_doller_data/Webstat_Export_20220928.csv'

us_dollar_data = pd.read_csv(us_dollar_data_file, sep=';', on_bad_lines='skip')
us_dollar_data = us_dollar_data.iloc[5:]

us_dollar_data.head(20)




,Titre :,Cours de l'or contre euro - 1er fixing à Londres,Cours de l'or contre dollar - 1er fixing à Londres
5,27/09/2022,"1696,76","1632,6"
6,26/09/2022,"1700,86",1647
7,25/09/2022,-,-
8,24/09/2022,-,-
9,23/09/2022,"1703,48","1661,45"
10,22/09/2022,"1692,92","1671,85"
11,21/09/2022,"1687,68","1674,45"
12,20/09/2022,"1665,56","1667,9"
13,19/09/2022,-,-
14,18/09/2022,-,-


In [87]:
us_dollar_data.iloc[:, 0] = pd.to_datetime(us_dollar_data.iloc[:, 0])


/tmp/ipykernel_70303/4189371360.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  us_dollar_data.iloc[:, 0] = pd.to_datetime(us_dollar_data.iloc[:, 0])
/tmp/ipykernel_70303/4189371360.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  us_dollar_data.iloc[:, 0] = pd.to_datetime(us_dollar_data.iloc[:, 0])


In [149]:
def read_curency_data():

    us_dollar_data_file = r'./data/us_doller_data/Webstat_Export_20220928.csv'

    us_dollar_data = pd.read_csv(us_dollar_data_file, sep=';', on_bad_lines='skip')
    us_dollar_data = us_dollar_data.iloc[5:]

    us_dollar_data.head(20)

    #us_dollar_data = us_dollar_data.dropna()

    us_dollar_data['date'] = us_dollar_data.iloc[:, 0]
    us_dollar_data['date'] =  pd.to_datetime(us_dollar_data['date'],  format='%d/%m/%Y')

    us_dollar_data['EUR'] = us_dollar_data.iloc[:, 1]
    us_dollar_data['EUR'] =us_dollar_data['EUR'].apply(lambda x: x.replace(',', '.'))
    us_dollar_data['EUR'] =us_dollar_data['EUR'].apply(lambda x: x.replace('-', ''))
    us_dollar_data['EUR'] = pd.to_numeric(us_dollar_data['EUR'])

    us_dollar_data['USD'] = us_dollar_data.iloc[:, 2]
    us_dollar_data['USD'] =us_dollar_data['USD'].apply(lambda x: x.replace(',', '.'))
    us_dollar_data['USD'] =us_dollar_data['USD'].apply(lambda x: x.replace('-', ''))
    us_dollar_data['USD'] = pd.to_numeric(us_dollar_data['USD'])

    us_dollar_data = us_dollar_data[['date', 'EUR', 'USD']]
    us_dollar_data = us_dollar_data.dropna()
    us_dollar_data =  us_dollar_data.reset_index(drop=True)
    
    return us_dollar_data


def df_taux_change(date = datetime(2016, 3, 1), currency = 'EUR'):
    
    if isinstance(date, datetime):
        currency_data = read_curency_data()
        values = currency_data[us_dollar_data['date'] == date][currency].tolist()
        
    else:
        raise NameError('Your should use datatime format for the date')

None


In [145]:
from datetime import datetime
#a_date = datetime.strptime('2022-09-22', format="%d/%m/%Y")
a_date = datetime(2016, 3, 1)
type (a_date)
values = us_dollar_data[us_dollar_data['date'] == a_date]['EUR'].tolist()
values

[1141.7]

In [148]:
isinstance(a_date, datetime)

True

### Tests

In [65]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [66]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [152]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... /tmp/ipykernel_70303/3818696156.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  french_data = french_data [~damaged_name_data]
FAIL
test_df_names_us (__main__.Session3Tests) ... 